In [1]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import tweepy
import os
import datetime, time
import re

In [2]:
home_dir = "/Users/christopherallison/.virtualenvs/py_twi/results"
save_dir = "/Users/christopherallison/Documents/Coding/Gephi/twitter"

# Enter your search queries here
search_queries = "rstats python".split()

### Functions for data prep

In [3]:
# Convert text to camel_case

first_cap_re = re.compile('(.)([A-Z][a-z]+)')
all_cap_re = re.compile('([a-z0-9])([A-Z])')

def convert(name):
    s1 = first_cap_re.sub(r'\1_\2', name)
    return all_cap_re.sub(r'\1_\2', s1).lower()

In [4]:
# Remove empty mentions created through dataframe list comprehension
# i.e, ['twitter_user_A', None]

def clean_mentions(scr, men):
    temp = []
    for s, m in zip(scr, men):
        if m:
            for i, v in enumerate(m):
                temp.append([s, m[i]])
        else:
            pass
        
    return temp
        

In [5]:
# Remove edges for reply_to_tweet where no reply exists
# Need to add code to assign weight for duplicate edges - multiple mentions or replies to from same person
# Will return list on len 3 - node, target, weight

def clean_edges(edge_list):
    clean_edges = []

    for e in edge_list:
        if e[1] == None:
            pass
        else:
            clean_edges.append(e)
            
    return clean_edges


In [6]:
# Check for duplicates in edges and compute number of repeats as edge weight

def sort_and_weight(edges):
    temp = {}

    for i in edges:
        try:
            temp[str(i)]['weight'] += 1
        except KeyError:
            temp[str(i)] = {}
            temp[str(i)]['node'] = i[0]
            temp[str(i)]['target'] = i[1]
            temp[str(i)]['weight'] = 1

    return temp

In [20]:
def twitter_data_to_graph(search_query):
    
    for sq in search_query:
        
        fname = open(os.path.join(home_dir, 'search_{}.json'.format(sq)))
        
        # Load the JSON file
        df = pd.read_json(fname)
        
        # Swap rows & columns
        df2 = df.transpose()
        
        # Change index from Timestamp to integer
        df2.index = [t.value // 1 for t in df2.index]
        df2.head()
        
        # Pull hashtags from dict
        new_hash = []

        for h in df2.hashtags:
            n = []
            for i in h:
                n.append(i['text'].lower())
            new_hash.append(n)
            
        # Pull user names from dict
        new_mentions = []

        for m in df2.user_mentions:
            n = []
            for i in m:
                n.append(i['screen_name'])
            new_mentions.append(n)
            
        # Set up lists for node & edge creation
        texts = [text for text in df2.text]
        dates = [d for d in df2.date]
        replies = [r for r in df2.reply_to_tweet]
        user_replies = [u for u in df2.reply_to]
        mentions = [m for m in new_mentions]
        screen_names = [sn for sn in df2.user_screen_name]
        hashtags = [h for h in new_hash]
        retweets = [r for r in df2.retweet_count]
        id_str = [i for i in df2.id_str]
        quote_id_str = [q for q in df2.quoted_status_id_str]
        coords = [c for c in df2.coordinates]
        favorites = [f for f in df2.favorite_count]
        
        # Create one node for each tweet with embedded data - Graph N
        for i, n, d, h, r, t, q, c, f  in zip(id_str, screen_names, dates, hashtags, retweets, texts, quote_id_str, coords, favorites):
            N.add_node(i, attr_dict={
                    'name':n,
                    'date':d,
                    'hashtags':str(h),
                    'retweets':r,
                    'text':t, 
                    #'quoted_id':q,
                    #'coords':c,
                    #'favorites':f,
                })
        
        # Create edges list for each tweet & reply_to_tweet
        
        edges_r = [[n, r] for n,r in zip(id_str, replies)]
        edges_q = [[n, q] for n,q in zip(id_str, quote_id_str)]
        
        # Combine edges and prep - expanding multiple @-mentions FREX
        edges_all = edges_r + edges_q
        edges_all = clean_edges(edges_all)
        
        # Create edge weight as number of repeat replies & @-mentions
        
        edge_dict = sort_and_weight(edges_all)
        
        # Add edges for @mentions and replies to users
        for e in edge_dict:
            N.add_edge(edge_dict[e]['node'],
                       edge_dict[e]['target'],
                       weight=edge_dict[e]['weight'])
            
        # Insert Data analysis here
        print("Nodes: {}, Edges: {}".format(len(N.nodes()), len(N.edges())))
        
    # Write N graph in gexf for Gephi
    file_name = "{}_tweets_graph_{}.gexf".format(
        convert("_".join(search_queries)),
        datetime.datetime.now())
    
    nx.write_gexf(N, os.path.join(save_dir, file_name))
    

In [21]:
# Set up Graph
N = nx.DiGraph()

In [22]:
twitter_data_to_graph(search_queries)

Nodes: 1028, Edges: 35
Nodes: 2027, Edges: 52


NameError: name 'df2' is not defined

1990-01-09 01:51:10.291136512  \
coordinates                                                  None   
date                                          2015-08-13 15:28:42   
entities        {'symbols': [], 'urls': [{'display_url': 'micr...   
favorite_count                                                  3   
hashtags        [{'text': 'python', 'indices': [30, 37]}, {'te...   

                                    1990-01-09 01:52:34.508730368  \
coordinates                                                  None   
date                                          2015-08-13 15:29:03   
entities        {'symbols': [], 'urls': [], 'media': [{'indice...   
favorite_count                                                  0   
hashtags              [{'text': 'python', 'indices': [100, 107]}]   

                                    1990-01-09 01:58:04.805947393  \
coordinates                                                  None   
date                                          2015-08-13 15:30:21   
entities        {'symbols': [], 'urls': [{'display_url': 'abiz...   
favorite_count                                                  0   
hashtags        [{'text': 'Python', 'indices': [0, 7]}, {'text...   

                                    1990-01-09 01:58:19.490238464  \
coordinates                                                  None   
date                                          2015-08-13 15:30:25   
entities        {'symbols': [], 'urls': [{'display_url': 'pypi...   
favorite_count                                                  0   
hashtags        [{'text': 'Euchner', 'indices': [57, 65]}, {'t...   

                                    1990-01-09 01:58:32.035368960  \
coordinates                                                  None   
date                                          2015-08-13 15:30:28   
entities        {'symbols': [], 'urls': [{'display_url': 'buff...   
favorite_count                                                  0   
hashtags        [{'text': 'python', 'indices': [90, 97]}, {'te...   

                                    1990-01-09 01:59:04.868261889  \
coordinates                                                  None   
date                                          2015-08-13 15:30:36   
entities        {'symbols': [], 'urls': [{'display_url': 'bit....   
favorite_count                                                  0   
hashtags        [{'text': 'python', 'indices': [68, 75]}, {'te...   

                                    1990-01-09 01:59:16.494966784  \
coordinates                                                  None   
date                                          2015-08-13 15:30:38   
entities        {'symbols': [], 'urls': [{'display_url': 'buff...   
favorite_count                                                  0   
hashtags        [{'text': 'programming', 'indices': [95, 107]}...   

                                    1990-01-09 01:59:37.286156288  \
coordinates                                                  None   
date                                          2015-08-13 15:30:43   
entities        {'symbols': [], 'urls': [{'display_url': 'buff...   
favorite_count                                                  2   
hashtags        [{'text': 'webdev', 'indices': [35, 42]}, {'te...   

                                    1990-01-09 01:59:55.560714240  \
coordinates                                                  None   
date                                          2015-08-13 15:30:48   
entities        {'symbols': [], 'urls': [{'display_url': 'neuv...   
favorite_count                                                  0   
hashtags        [{'text': 'Python', 'indices': [14, 21]}, {'te...   

                                    1990-01-09 02:01:07.341893632  \
coordinates                                                  None   
date                                          2015-08-13 15:31:05   
entities        {'symbols': [], 'urls': [], 'hashtags': [{'tex...   
favorite_count           

coordinates                 date  \
1990-01-09 01:51:10.291136512        None  2015-08-13 15:28:42   
1990-01-09 01:52:34.508730368        None  2015-08-13 15:29:03   
1990-01-09 01:58:04.805947393        None  2015-08-13 15:30:21   
1990-01-09 01:58:19.490238464        None  2015-08-13 15:30:25   
1990-01-09 01:58:32.035368960        None  2015-08-13 15:30:28   

                                                                        entities  \
1990-01-09 01:51:10.291136512  {'symbols': [], 'urls': [{'display_url': 'micr...   
1990-01-09 01:52:34.508730368  {'symbols': [], 'urls': [], 'media': [{'indice...   
1990-01-09 01:58:04.805947393  {'symbols': [], 'urls': [{'display_url': 'abiz...   
1990-01-09 01:58:19.490238464  {'symbols': [], 'urls': [{'display_url': 'pypi...   
1990-01-09 01:58:32.035368960  {'symbols': [], 'urls': [{'display_url': 'buff...   

                              favorite_count  \
1990-01-09 01:51:10.291136512              3   
1990-01-09 01:52:34.508730368              0   
1990-01-09 01:58:04.805947393              0   
1990-01-09 01:58:19.490238464              0   
1990-01-09 01:58:32.035368960              0   

                                                                        hashtags  \
1990-01-09 01:51:10.291136512  [{'text': 'python', 'indices': [30, 37]}, {'te...   
1990-01-09 01:52:34.508730368        [{'text': 'python', 'indices': [100, 107]}]   
1990-01-09 01:58:04.805947393  [{'text': 'Python', 'indices': [0, 7]}, {'text...   
1990-01-09 01:58:19.490238464  [{'text': 'Euchner', 'indices': [57, 65]}, {'t...   
1990-01-09 01:58:32.035368960  [{'text': 'python', 'indices': [90, 97]}, {'te...   

                                           id_str lang quoted_status reply_to  \
1990-01-09 01:51:10.291136512  631849870291136512   en         False     None   
1990-01-09 01:52:34.508730368  631849954508730368   en         False     None   
1990-01-09 01:58:04.805947393  631850284805947393   en         False     None   
1990-01-09 01:58:19.490238464  631850299490238464   de         False     None   
1990-01-09 01:58:32.035368960  631850312035368960   de         False     None   

                              reply_to_tweet retweet_count  \
1990-01-09 01:51:10.291136512           None             0   
1990-01-09 01:52:34.508730368           None            14   
1990-01-09 01:58:04.805947393           None             0   
1990-01-09 01:58:19.490238464           None             1   
1990-01-09 01:58:32.035368960           None             0   

                                                                            text  \
1990-01-09 01:51:10.291136512  Microsoft teaching how to use #python and #Dja...   
1990-01-09 01:52:34.508730368  RT @MuseumsInspire: Moving robots in the 'Pi P...   
1990-01-09 01:58:04.805947393  #Python-gdal create geotiff from array with co...   
1990-01-09 01:58:19.490238464  RT @dajool_com: Hat jemand ein Electronic-Key-...   
1990-01-09 01:58:32.035368960  Python Library for Amadeus Travel Innovation S...   

                                                                            urls  \
1990-01-09 01:51:10.291136512  [{'display_url': 'microsoftvirtualacademy.com/...   
1990-01-09 01:52:34.508730368                                                 []   
1990-01-09 01:58:04.805947393  [{'display_url': 'abizy.com/p/view.html?ur…', ...   
1990-01-09 01:58:19.490238464  [{'display_url': 'pypi.python.org/pypi/eks/0.1...   
1990-01-09 01:58:32.035368960  [{'display_url': 'buff.ly/1P8q30m', 'indices':...   

                                                                            user  \
1990-01-09 01:51:10.291136512  {'contributors_enabled': False, 'notifications...   
1990-01-09 01:52:34.508730368  {'contributors_enabled': False, 'notifications...   
1990-01-09 01:58:04.805947393  {'contributors_enabled': False, 'notifications...   
1990-01-09 01:58:19.490238464  {'contributors_enabled': False, 'notifications...   
1990-01-09 01:58:32.035368960  {'contributor

Just noticed that date are incorrect - these are the tweet ID's that have been converted by accident

## Scripts for setting up graph of all tweets

[('631734304524005376',
  {'date': '2015-08-13 07:49:29',
   'hashtags': "['Python', 'optimization', 'performance', 'funny', 'Twitter']",
   'name': 'slacknux',
   'retweets': 0,
   'text': 'Good talk https://t.co/iSZ7GS3BxH #Python #optimization #performance #funny #Twitter'}),
 ('631565819781218304',
  {'date': '2015-08-12 20:40:00',
   'hashtags': "['Python', 'edtech']",
   'name': 'Fuliski',
   'retweets': 3,
   'text': 'RT @Don_Watkins: Using #Python &amp; Raspberry Pi to communicate w/ Lego Mindstorms EV3 | https://t.co/v5LfotVS2o #edtech @rickweinberg @philsh…'}),
 ('631569161991376896',
  {'date': '2015-08-12 20:53:16',
   'hashtags': "['IPython', 'Python']",
   'name': 'ppblt',
   'retweets': 24,
   'text': 'RT @randal_olson: Big changes w/ the latest #IPython 4.0 release.\n\n"ipython notebook" no longer works?\n\nhttp://t.co/j0MAe0w7fy #Python http…'}),
 ('631567373972058112',
  {'date': '2015-08-12 20:46:10',
   'hashtags': "['python']",
   'name': 'artwisanggeni',
   'retwe

[['kenwalger', 'kennethlove'],
 ['groundwalkergmb', 'jeanhuguesroy'],
 ['uogbuji', 'arh'],
 ['j4pe_', 'eric_pommereau'],
 ['Don_Watkins', 'rickweinberg'],
 ['HarriSrivastav', 'algoritmic'],
 ['HarriSrivastav', 'lonriesberg'],
 ['ishamyyl', 'climagic'],
 ['HarriSrivastav', 'artwisanggeni'],
 ['HarriSrivastav', 'Proximity_ltd'],
 ['HarriSrivastav', 'MarketACourse'],
 ['HarriSrivastav', 'BestSQL'],
 ['HarriSrivastav', 'ElaMoscicka'],
 ['HarriSrivastav', 'artwisanggeni'],
 ['HarriSrivastav', 'nodenow'],
 ['HarriSrivastav', '_testanic'],
 ['_organicit', 'pcgeek86'],
 ['melindathrasher', 'djangogirls'],
 ['ryanteck', 'MuseumsInspire'],
 ['ryanteck', 'computermuseum'],
 ['ryanteck', 'Raspberry_Pi'],
 ['lgtr', 'dajool_com'],
 ['hayza11', 'googletricks19'],
 ['kenwalger', 'kennethlove'],
 ['isurunix', 'nodenow'],
 ['MikeHerman', 'pycoders'],
 ['MikeHerman', 'Podcast__init__'],
 ['MikeHerman', 'jessicamckellar'],
 ['joebalalaika', 'RealPython'],
 ['mr_brejoc', 'lgtr'],
 ['Patty_1982', 'MuseumsIn

[('funKidsSTEM', 'atcsapp'),
 ('BobBodily', 'PlaceILive'),
 ('BobBodily', 'LoralynTaylor'),
 ('BobBodily', 'sepirdata'),
 ('pdr2002', 'rae_simon'),
 ('robbyki', 'Hancock_JohnD'),
 ('Jordanmlima', 'randal_olson'),
 ('emilyhwilson', 'fredmcconnell'),
 ('emilyhwilson', 'OrbitalMechs'),
 ('cmarschner', 'randal_olson'),
 ('BitterMoa', 'moaimx'),
 ('BitterMoa', 'GIRE_mx'),
 ('BitterMoa', 'Data4mx'),
 ('Niels_Bremen', 'flowingdata'),
 ('Niels_Bremen', 'randal_olson'),
 ('GoClickSpree', 'gregmancel'),
 ('GoClickSpree', 'DataScienceCtrl'),
 ('jknowles', 'HollyLynnMck'),
 ('jknowles', 'Biff_Bruise'),
 ('jknowles', 'EamonCaddigan'),
 ('jknowles', 'sweetspot'),
 ('matlabulous', 'NatGeo'),
 ('matlabulous', 'visualisingdata'),
 ('matlabulous', 'NadiehBremer'),
 ('santoshosu', 'DecisionViz'),
 ('santoshosu', 'alteryx'),
 ('BioMickWatson', 'randal_olson'),
 ('MAGISTR_OM', 'SkytreeHQ'),
 ('MAGISTR_OM', 'KirkDBorne'),
 ('GerryWieder', 'vjo'),
 ('luzumuyok', 'Ronald_vanLoon'),
 ('yayayarndiva', 'bayrepor

2628